In [2]:
import json
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.graphs import Neo4jGraph
from langchain_core.documents import Document
from sympy.physics.units import temperature
from tornado.escape import utf8

load_dotenv()
try:
    graph=Neo4jGraph()
except Exception as e:
    print(e)
    exit()

llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash',temperature=0)
allowed_nodes = ["Person", "Organization", "Location", "Penalty", "Date", "Offense", "Condition"]
allowed_rels = ["HAS_PENALTY", "APPLIES_TO", "LOCATED_IN", "DEFINED_AS", "EXCLUDES", "ENFORCED_BY", "MENTIONS"]
llm_transformer=LLMGraphTransformer(llm=llm,allowed_nodes=allowed_nodes,allowed_relationships=allowed_rels)

In [3]:
'''graph.query("MATCH (n) DETACH DELETE n")'''

[]

In [15]:
with open("cpa_anchored.json","r",encoding="utf-8") as f:
    data=json.load(f)
    for chapter in data:
        chapter_name=chapter["chapter_name"]
        graph.query("MERGE (c:Chapter {id : $id})", params={'id': chapter_name})
        for section in chapter["sections"]:
            section_id = str(section["section_id"])
            section_title= section["title"]
            text=section["title"]+section["original_content"]
            graph.query('''MATCH (c:Chapter {id : $cid}) MERGE (s:Section {id: $sid}) SET s.title=$title, s.text=$text MERGE (c)-[:CONTAINS]->(s)''',params={'cid': chapter_name,'sid':section_title, 'title': section_title, 'text': text})
            doc = Document(page_content=text)
            graph_extracted=llm_transformer.convert_to_graph_documents([doc])
            if graph_extracted:
                graph_docs=graph_extracted[0]
                graph.add_graph_documents([graph_docs])
                for node in graph_docs.nodes:
                    graph.query(
                    '''MATCH (s:Section {id :$id})
                    MATCH (n) WHERE elementId(n)=$node_id
                    MERGE (s)-[:CONTAINS]->(n)''', params={'node_id':node.id, 'id':section_id}
                    )

